In [2]:
!pip install pandas

     ---------------------------------------- 10.6/10.6 MB 9.2 MB/s eta 0:00:00


In [4]:
!pip install nltk

     ---------------------------------------- 1.5/1.5 MB 5.6 MB/s eta 0:00:00
     -------------------------------------- 307.0/307.0 KB 6.3 MB/s eta 0:00:00
     ------------------------------------- 262.0/262.0 KB 16.8 MB/s eta 0:00:00
     ---------------------------------------- 96.6/96.6 KB 5.4 MB/s eta 0:00:00
     ---------------------------------------- 78.4/78.4 KB 4.5 MB/s eta 0:00:00


In [27]:
!pip install sklearn

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 7.2/7.2 MB 4.7 MB/s eta 0:00:00
     ---------------------------------------- 37.0/37.0 MB 4.5 MB/s eta 0:00:00
Using legacy 'setup.py install' for sklearn, since package 'wheel' is not installed.
  Running setup.py install for sklearn: started
  Running setup.py install for sklearn: finished with status 'done'


In [1]:
import pandas as pd
import numpy as np

import nltk

import re #регулярные выражения
import math
from collections import Counter
import requests
import time
alphabet = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'

In [2]:
BYTES_TO_MB_DIV = 0.000001
def print_memory_usage_of_data_frame(df):
    mem = round(df.memory_usage().sum() * BYTES_TO_MB_DIV, 3) 
    print("Memory usage is " + str(mem) + " MB")

In [3]:
nltk.download('stopwords') # to use stopwords
nltk.download('punkt') # to use word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wequalwo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wequalwo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords as stopwords
from nltk.tokenize import word_tokenize
 
example_sent = """Это простое предложение показывает фильтрацию на стоп-слова"""
 
stop_words = set(stopwords.words('russian'))
 
word_tokens = word_tokenize(example_sent)
 
filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
 
filtered_sentence = []
 
for w in word_tokens:
    if w not in stop_words:
        filtered_sentence.append(w)
 
print(word_tokens)
print(filtered_sentence)

['Это', 'простое', 'предложение', 'показывает', 'фильтрацию', 'на', 'стоп-слова']
['Это', 'простое', 'предложение', 'показывает', 'фильтрацию', 'стоп-слова']


In [5]:
from nltk.corpus import stopwords
stopwords.words('russian')

['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'на

**Создадим функции, позволяющие генерировать следующие ошибки в словах:**

1.   **Пропуск буквы:** ${резонанс - рзонанс}$
2.   **Дублирование буквы:** ${резонанс - реезонанс}$
3.   **Перестановка букв:** ${резонанс - рзеонанс}$
4.   **Опечатка:** ${резонанс - ркзонанс}$
5.   **Некоторые орфографические ошибки:**  ${резонанс - ризонанс}$


In [6]:
# все ошибки на расстоянии 2 (расстояние Левинштейна)
def distance2(word):
  return {e2 for e1 in distance1(word) if e1 for e2 in distance1(e1)}

# все ошибки на расстоянии 1 (расстояние Левинштейна)
def distance1(word):
    pairs      = splits(word)
    transposes = [a+b[1]+b[0]+b[2:]  for (a,b) in pairs if len(b)>1]                           # перестановки
    replaces   = [a+c+b[1:]          for (a,b) in pairs if b for c in replaces_set[b[0]] if b] # замены: 
                                                                                                        # опечатки, 
                                                                                                        # пропуски и дублирования букв, 
                                                                                                        # орфографические ошибки

    last_replaces = [word[0:-1] + c for c in replaces_set[word[-1]]]                           # замены в конце слова
    return set(transposes + replaces + last_replaces)

def splits(word):
    return [(word[:i], word[i:])
            for i in range (len(word)+1)]

# список возможных замен в слове 'с', 'в', 'у', 'к', 'е', 'п', 'м', 'о', 'aa', ''
replaces_set = pd.Series([          
               ['с', 'в', 'у', 'к', 'е', 'п', 'м', 'о', 'аа', ''],
               ['ь', 'о', 'л', 'д', 'ж', 'ю', 'п', 'бб', ''],
               ['ч', 'ы', 'ц', 'у', 'к', 'а', 'с', 'вв', 'ф', ''],
               ['р', 'н', 'ш', 'л', 'о', 'гг', 'к', ''],
               ['б', 'л', 'ш', 'щ', 'з', 'ж', 'ю', 'дд', 'т', ''],
               ['а', 'к', 'н', 'р', 'п', 'ее', 'и', 'ё', 'о', ''],
               ['ёё', 'й', 'йо', 'е', ''],
               ['ю', 'д', 'щ', 'з', 'х', 'э', 'жж', 'ш', ''],
               ['д', 'щ', 'х', 'э', 'ж', 'зз', 'с', ''],
               ['м', 'а', 'п', 'р', 'т', 'ии', 'е', 'ы', ''],
               ['ц', 'ы', 'ф', 'йй',''],
               ['а', 'в', 'у', 'е', 'п', 'п', 'а', 'кк', 'г', ''],
               ['ь', 'о', 'г', 'ш', 'щ', 'д', 'б', 'лл', ''],
               ['с', 'а', 'п', 'и', 'мм', 'н', ''],
               ['р', 'п', 'е', 'г', 'о', 'нн', 'м', ''],
               ['т', 'р', 'н', 'г', 'ш', 'л', 'ь', 'оо', 'а', 'е', 'ё', 'у', ''],
               ['м', 'а', 'к', 'е', 'н', 'р', 'и', 'пп', 'б', ''],
               ['и', 'п', 'е', 'н', 'г', 'о', 'т', 'рр', 'р', ''],
               ['ч', 'в', 'а', 'м', 'сс', 'з', ''],
               ['и', 'п', 'р', 'о', 'ь', 'тт', 'д', ''],
               ['в', 'ы', 'ц', 'к', 'а', 'уу', 'ю', 'о', ''],
               ['я', 'ч', 'ы', 'ц', 'й', 'фф', 'в', ''],
               ['ж', 'з', 'ъ', 'э', 'хх', ''],
               ['ы', 'ф', 'й', 'у', 'в', 'ы', 'цц', 'ч', ''],
               ['я', 'ф', 'ы', 'в', 'с', 'чч', ''],
               ['л', 'о', 'г', 'щ', 'д', 'шш', 'щ', 'шь', 'щь', 'ж', ''],
               ['л', 'ш', 'з', 'ж', 'д', 'щщ', 'щь', 'шь', 'ж', ''],
               ['э', 'х', 'ъъ', 'ь', ''],
               ['ч', 'я', 'ф', 'й', 'ц', 'у', 'в', 'и', 'ыы', ''],
               ['т', 'о', 'л', 'б', 'ьь', 'ъ', ''],
               ['ж', 'з', 'х', 'ъ', ''],
               ['б', 'л', 'д', 'ж', 'э', 'у', 'йу', ''],
               ['ф', 'ы', 'ч', 'яя', 'а', 'йа', ''],
               ], 
               index = ['а','б','в','г', 'д', 'е', 'ё', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я'])


In [114]:
import codecs
with codecs.open('top10000.txt', 'r', encoding='utf-8') as file:
    TEXT = file.read().replace('\n', ' ')


def tokens(text):
    return re.findall(r'[а-я]+', text.lower())
list_of_words = tokens(TEXT)

# удалим все слова, короче 3 символов (потому что слова из 2х символов неинформативны):
tmp = [w for w in list_of_words if len(w) > 2]
list_of_words = tmp

del list_of_words[500:]

#print(list_of_words)
print(len(list_of_words))

500


In [122]:
words = []
for item in list_of_words:
  words.append([item, distance1(item).union(distance2(item))])# ??? как оптимизировать - неясно

In [124]:
def _tokenize(word):
  return " ".join([a + a for a in word])
_tokenize('привет')

'пп рр ии вв ее тт'

In [125]:
tokenized = []
to_vec = []
for w in words:
  tokens = []
  for errors in w[1]:
    st = _tokenize(errors)
    tokens.append(st)
    to_vec.append(st)
  tokenized.append([w[0], tokens])

In [126]:
# print(to_vec[400:450])
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = to_vec
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

In [127]:
# найденные токены:
vectorizer.get_feature_names_out()

array(['аа', 'бб', 'вв', 'гг', 'дд', 'ее', 'жж', 'зз', 'ии', 'йй', 'кк',
       'лл', 'мм', 'нн', 'оо', 'пп', 'рр', 'сс', 'тт', 'уу', 'фф', 'хх',
       'цц', 'чч', 'шш', 'щщ', 'ъъ', 'ыы', 'ьь', 'ээ', 'юю', 'яя', 'ёё'],
      dtype=object)

In [128]:
df = pd.DataFrame(X.toarray())

In [129]:
labels = []
_words = []
for i in range(0, len(tokenized)):
  for j in range(0, len(tokenized[i][1])):
    labels.append(i)
    _words.append(tokenized[i][0])
df["label"] = labels

In [132]:
df

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,label
0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.751400,0.0,0.0,0.0,0.000000,0
1,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.617764,0.000000,0.0,0.0,0.0,0.000000,0
2,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.736919,0
3,0.273837,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0
4,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.369858,0.0,0.0,0.0,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884053,0.251631,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.281763,0.352994,...,0.0,0.0,0.0,0.395686,0.000000,0.0,0.0,0.0,0.000000,499
884054,0.000000,0.0,0.232907,0.0,0.0,0.0,0.0,0.0,0.212751,0.266535,...,0.0,0.0,0.0,0.298770,0.000000,0.0,0.0,0.0,0.000000,499
884055,0.269069,0.0,0.329834,0.0,0.0,0.0,0.0,0.0,0.000000,0.377457,...,0.0,0.0,0.0,0.423107,0.000000,0.0,0.0,0.0,0.000000,499
884056,0.000000,0.0,0.650069,0.0,0.0,0.0,0.0,0.0,0.296906,0.371965,...,0.0,0.0,0.0,0.416951,0.000000,0.0,0.0,0.0,0.000000,499


In [133]:
from sklearn.model_selection import train_test_split
from sklearn.metrics  import f1_score, accuracy_score


from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [135]:
#first we split our data into input and output
# y is the output and is stored in "Class" column of dataframe
# X contains the other columns and are features or input
# train = df
# y = train.label
# train.drop(['label'], axis = 1, inplace=True)
# X = train
# Now we split the dataset in train and test part
# here the train set is 75% and test set is 25%

# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 2)

In [134]:
# CL = DecisionTreeClassifier()
# CL.fit(X_train, y_train)

In [551]:
# pred = CL.predict(X_test)

In [142]:
def convert_to_sparse_pandas(df, exclude_columns = []):
    df = df.copy()
    exclude_columns = set(exclude_columns)

    for (columnName, columnData) in df.iteritems():
        if columnName in exclude_columns:
            continue
        df[columnName] = pd.arrays.SparseArray(columnData.values)

    return df

df_sparse = convert_to_sparse_pandas(df, exclude_columns = ["label"])
display(df_sparse.dtypes)
# print_memory_usage_of_data_frame(data_one_hot_sparse)

0        Sparse[float64, nan]
1        Sparse[float64, nan]
2        Sparse[float64, nan]
3        Sparse[float64, nan]
4        Sparse[float64, nan]
5        Sparse[float64, nan]
6        Sparse[float64, nan]
7        Sparse[float64, nan]
8        Sparse[float64, nan]
9        Sparse[float64, nan]
10       Sparse[float64, nan]
11       Sparse[float64, nan]
12       Sparse[float64, nan]
13       Sparse[float64, nan]
14       Sparse[float64, nan]
15       Sparse[float64, nan]
16       Sparse[float64, nan]
17       Sparse[float64, nan]
18       Sparse[float64, nan]
19       Sparse[float64, nan]
20       Sparse[float64, nan]
21       Sparse[float64, nan]
22       Sparse[float64, nan]
23       Sparse[float64, nan]
24       Sparse[float64, nan]
25       Sparse[float64, nan]
26       Sparse[float64, nan]
27       Sparse[float64, nan]
28       Sparse[float64, nan]
29       Sparse[float64, nan]
30       Sparse[float64, nan]
31       Sparse[float64, nan]
32       Sparse[float64, nan]
label     

In [143]:
y = df['label']
X = df[df.columns.difference(['label'])]
y_sparse = df_sparse['label']
X_sparse = df_sparse[df_sparse.columns.difference(['label'])]
print_memory_usage_of_data_frame(X)
print_memory_usage_of_data_frame(X_sparse)

Memory usage is 233.391 MB
Memory usage is 350.087 MB


In [144]:
from scipy.sparse import lil_matrix

def data_frame_to_scipy_sparse_matrix(df):
    arr = lil_matrix(df.shape, dtype = np.float32)
    for i, col in enumerate(df.columns):
        ix = df[col] != 0
        arr[np.where(ix), i] = 1
    return arr.tocsr()

def get_csr_memory_usage(matrix):
    mem = (X_csr.data.nbytes + X_csr.indptr.nbytes + X_csr.indices.nbytes) * BYTES_TO_MB_DIV
    print("Memory usage is " + str(mem) + " MB")
    
y_csr = y_sparse
X_csr = data_frame_to_scipy_sparse_matrix(X_sparse)

In [145]:
print('X:')
print_memory_usage_of_data_frame(X)
print('X_sparse:')
print_memory_usage_of_data_frame(X_sparse)
print('X_csr:')
get_csr_memory_usage(X_csr)

X:
Memory usage is 233.391 MB
X_sparse:
Memory usage is 350.087 MB
X_csr:
Memory usage is 46.89978 MB


In [ ]:
model = DecisionTreeClassifier()

vector_dict = {'Pandas dataframe': [X,y],
    'Sparse pandas dataframe': [X_sparse, y_sparse],
    'Scipy sparse matrix': [X_csr, y_csr]
    }

start = time.time()
X_train, X_test, y_train, y_test = train_test_split(X_csr, y_csr, test_size = 0.3, random_state = 42)
end = time.time()

duration = round(end - start, 2)
print("Train-test split: " + str(duration) + " secs")

start = time.time()
model.fit(X_train, y_train)

end = time.time()
duration = round(end - start, 2)
print("Training: " + str(duration) + " secs")
print("\n")

Train-test split: 0.24 secs


In [111]:
start = time.time()
pred = model.predict(X_test)
end = time.time()
duration = round(end - start, 2)
print("Testing: " + str(duration) + " secs")
print("\n")

Testing: 55.93 secs




In [112]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

print(accuracy_score(y_test, pred))
print(f1_score(y_test, pred, average = 'micro'))
 
print(mean_absolute_error(y_test, pred))

0.013475827849223401
0.013475827849223401
3471.845461988808
